# Web Scraping del sito Kijiji



## Import delle librerie


In [1]:
import requests
import bs4
import csv
from tqdm import tqdm_notebook as tqdm
import json
import pprint
import re

## Definizione url di riferimento

Per semplificare il progetto la ricerca riguarderà tutti gli annunci trovati filtrando per regione Lombardia e prodotto iPhone.


In [2]:
webpage = f"https://www.kijiji.it/annunci-lombardia/iphone/?p=1"

Cerco tutti gli annunci nella pagina:

In [3]:
response = requests.get(webpage)
doc = bs4.BeautifulSoup(response.text, "lxml")
annunci = doc.find_all('li', class_="result")

In [4]:
len(annunci)

26

Ci sono 26 annunci nella pagina!

Test del codice per lo scarico delle prime informazioni nella pagina di ricerca degli annunci:

In [5]:
annunci_list = []
for annuncio in annunci:
    try:
        titolo = annuncio.select("h3.title > a")[0].text.replace("\n", "").strip()
    except:
        continue
    url = annuncio["data-href"]
    annuncio_id = annuncio["data-id"]
    try:
        prezzo = annuncio.select("div.item-content > h4.price")[0].text.replace("\n", "").replace("€","").strip()
    except:
        prezzo = "N/A"
    try:
        data = annuncio.select("div.item-content > p.timestamp")[0].text
    except:
        data = "N/A"
    try:
        paese = annuncio.select("div.item-content > p.locale")[0].text
    except:
        paese = "N/A"
    annunci_list.append({'id': annuncio_id, 'titolo': titolo, 'link': url, 
                         'prezzo': prezzo, 'data': data, 'paese' :paese})
    
print(annunci_list[0])

{'id': '138808297', 'titolo': 'Iphone xs max 512GB', 'link': 'https://www.kijiji.it/annunci/cellulari-e-accessori/como-annunci-como/iphone-xs-max/138808297', 'prezzo': '890', 'data': 'Oggi, 09:04', 'paese': 'Como'}


Ok, ora posso definire la funzione per il parsing generico degli annunci:

In [6]:

def parse_annuncio(annuncio):
    try:
        titolo = annuncio.select("h3.title > a")[0].text.replace("\n", "").strip()
    except:
        return None
    url = annuncio["data-href"]
    annuncio_id = annuncio["data-id"]
    try:
        prezzo = annuncio.select("div.item-content > h4.price")[0].text.replace("\n", "").replace("€","").strip()
    except:
        prezzo = "N/A"
    try:
        data = annuncio.select("div.item-content > p.timestamp")[0].text
    except:
        data = "N/A"
    try:
        paese = annuncio.select("div.item-content > p.locale")[0].text
    except:
        paese = "N/A"
    return {'id': annuncio_id, 'titolo': titolo, 'link': url, 
                         'prezzo': prezzo, 'data': data, 'paese' :paese}


In un ciclo for gestisco il download degli annunci per ogni pagina:

In [7]:
annunci_list = []
for num in tqdm(range(1,11)):
    webpage = f"https://www.kijiji.it/annunci-lombardia/iphone/?p={num}"
    response = requests.get(webpage)
    doc = bs4.BeautifulSoup(response.text, "lxml")
    annunci = doc.find_all('li', class_="result")
    for annuncio in annunci:
        annunci_list.append(parse_annuncio(annuncio))

print(len(annunci_list))
        

A Jupyter Widget


26


## Come terminare lo scraping?

Problema: *quando mi fermo?*
- cerco all'interno della pagina il numero totale di pagine


In [8]:
firstpage = f"https://www.kijiji.it/annunci-lombardia/iphone/?p=1"
firstresponse = requests.get(firstpage)
firstdoc = bs4.BeautifulSoup(firstresponse.text, "lxml")
pagecount = int(firstdoc.select("#pagination > div > a.last-page")[0].text)
print(pagecount)

45


In [9]:
annunci_list = []
annunci_letti = 0
num = 0
for num in tqdm(range(1,2)):
    num = num + 1
    webpage = f"https://www.kijiji.it/annunci-lombardia/iphone/?p={num}"
    response = requests.get(webpage)
    doc = bs4.BeautifulSoup(response.text, "lxml")
    annunci = doc.find_all('li', class_="result")
    for annuncio in annunci:
        try:
            annuncio_parsed = parse_annuncio(annuncio)
            if annuncio_parsed is not None:
                annunci_list.append(annuncio_parsed)
        except:
            pass
    annunci_letti = len(annunci_list)

A Jupyter Widget

## Pandas
Utilizzo la libreria Pandas per salvare i risultati in un file csv:

In [10]:
import pandas as pd
ds_annunci = pd.DataFrame(annunci_list)
ds_annunci.set_index("id")
ds_annunci.head()

,data,id,link,paese,prezzo,titolo
0,"Oggi, 09:04",138808297,https://www.kijiji.it/annunci/cellulari-e-acce...,Como,890,Iphone xs max 512GB
1,"Ieri, 11:43",138705039,https://www.kijiji.it/annunci/cellulari-e-acce...,Maciachini / Zara / Niguarda,400,IPhone 8 (2018) Nero - 64GB
2,"17 giugno, 20:13",139532342,https://www.kijiji.it/annunci/cellulari-e-acce...,Sempione / Certosa,270,IPhone 7 32 Gb Gold
3,"17 giugno, 20:03",139529157,https://www.kijiji.it/annunci/cellulari-e-acce...,Mezzago,1.150,IPhone XS 512 gb colore silver nuovo
4,"17 giugno, 20:03",139529109,https://www.kijiji.it/annunci/cellulari-e-acce...,Mezzago,900,IPhone XS 64 gb space gray nuovo


In [11]:
ds_annunci.to_csv("./kijiji_annunci.csv")

## Le pagine degli annunci ###
Ora l'obiettivo è scaricare i dettagli dei singoli annunci e le foto.
# Geocoding
Per avere una maggiore precisione riguardo all'informazione geografica degli annunci, richiamo il servizio di geocoding in modo da ottenere una geolocalizzazione più strutturata.


In [12]:
import json
# apro il file csv
import pandas as pd
ds_annunci = pd.read_csv("./kijiji_annunci.csv",encoding = "ISO-8859-1", index_col="id")
ds_annunci.head()

,Unnamed: 0,data,link,paese,prezzo,titolo
id,,,,,,
138808297,0,"Oggi, 09:04",https://www.kijiji.it/annunci/cellulari-e-acce...,Como,890.00,Iphone xs max 512GB
138705039,1,"Ieri, 11:43",https://www.kijiji.it/annunci/cellulari-e-acce...,Maciachini / Zara / Niguarda,400.00,IPhone 8 (2018) Nero - 64GB
139532342,2,"17 giugno, 20:13",https://www.kijiji.it/annunci/cellulari-e-acce...,Sempione / Certosa,270.00,IPhone 7 32 Gb Gold
139529157,3,"17 giugno, 20:03",https://www.kijiji.it/annunci/cellulari-e-acce...,Mezzago,1.15,IPhone XS 512 gb colore silver nuovo
139529109,4,"17 giugno, 20:03",https://www.kijiji.it/annunci/cellulari-e-acce...,Mezzago,900.00,IPhone XS 64 gb space gray nuovo


Per ogni annuncio vado a richiamare il link e a procedere con lo scarico dalla pagina di dettaglio:

In [15]:
dettagli = []
for annuncio_id, annuncio in tqdm(ds_annunci.iterrows(), total=ds_annunci.shape[0]):
    dettagli_row = {}
    link = annuncio["link"]
    response = requests.get(link)
    doc = bs4.BeautifulSoup(response.text, "lxml")
    try:
        descrizione = doc.select("article.vip__carousel > div.box__content > div > p.vip__text-description")[0].text.replace('\n', ' ').strip()
    except:
        continue
    try:
        nome_venditore = doc.select("article.user > div.media__body > div.title")[0].text.replace('\n', ' ').strip()
    except:
        nome_venditore = "N/A"
    try: 
        phone = doc.select("h3.modal-phone__text")[0].text.replace('\n', ' ').strip().replace("+", "00")
    except:
        phone = 'N/A'
    try: 
        userId = doc.select("div.user__bottom > a")[0]["href"].split("/")[-1]
    except:
        userId = 'N/A'
    id_ad = doc.select("li.breadcrumbs__leaf > strong")[0].text.replace('\n', ' ').strip()
    try:
        location = doc.select("div.vip__location > div > div > span")[0].text.replace('\n', ' ').strip()
    except: 
        location1 = doc.select("div.vip__location")[0]
        location =location1.select("div > span")[0].text
    dettagli_row.update({'descrizione': descrizione,  'nome_venditore': nome_venditore,
                        'phone': phone, 'userId':userId, 'id_ad':id_ad, 'location': location})
    details = doc.select("section.vip__details > dl")
    new_table = pd.DataFrame(columns=range(0,2), index = [0]) # I know the size 
    row_marker = 0
    for row in details:
        #column_marker = 0
        label = row.find_all('dt')[0].text.replace('\n', ' ').strip()
        value = row.find_all('dd')[0].text.replace('\n', ' ').strip()
        dettagli_row.update({label:value})
    ## geocoding
    lat = ""
    lon = ""
    try:
        key = "T4eqDjtnpzWsfeMBZgKAqKobvcICurpU"
        geocode_url = f"http://www.mapquestapi.com/geocoding/v1/address?key={key}&location={dettagli_row.get('location')}"
        response = requests.get(geocode_url)
        geo = json.loads(response.text)
        lat = geo['results'][0]['locations'][0]['latLng']['lat']
        lon = geo['results'][0]['locations'][0]['latLng']['lng']
    except:
        pass
    #scarico le immagini
    docImg = doc.select("section.vip__body > article.vip__carousel > div.carousel-slide ")[0]
    imgs = docImg.find_all("img")
    i = 0
    for img in imgs:
        src = img['src']
        img_file = requests.get(src, stream=True)
        if img_file.status_code == 200:
            with open("./img_kijiji/img_" + str(id_ad) + "_" + str(i) + ".jpg", 'wb') as f:
                f.write(img_file.content)
            i = i+1
    dettagli_row.update({"lat": lat, "lon": lon})
    dettagli.append(dettagli_row)

A Jupyter Widget

In [16]:
ds_dettagli = pd.DataFrame(dettagli)
#ds_dettagli.set_index("id")
ds_dettagli.head()

,Marca,Tipologia,descrizione,id_ad,lat,location,lon,nome_venditore,phone,userId
0,Apple,Cellulari,"Iphone xs max da 512 gb , acquistato alla medi...",138808297,45.810681,Como (Como),9.086303,Getuar,00393347324251,26440510
1,Apple,Cellulari,"Vendo iPhone 8 regalato a gennaio 2018, tenuto...",138705039,45.466797,Milano (Milano),9.190498,Filippo,00393408674227,26431725
2,Apple,Cellulari,"Vendo iPhone 7 32 Gb Gold, tenuto sempre con l...",139532342,45.466797,Milano (Milano),9.190498,Adriano,00393409554474,10956731
3,Apple,Cellulari,In vendita un iPhone XS 512 gb silver nuovo si...,139529157,45.616030,Mezzago (Monza/Brianza),9.447920,Dina,N/A,3532037
4,Apple,Cellulari,Vendo un iPhone XS 64 gb colore space gray nuo...,139529109,45.616030,Mezzago (Monza/Brianza),9.447920,Dina,N/A,3532037


In [17]:
ds_dettagli.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 10 columns):
Marca             5 non-null object
Tipologia         5 non-null object
descrizione       5 non-null object
id_ad             5 non-null object
lat               5 non-null float64
location          5 non-null object
lon               5 non-null float64
nome_venditore    5 non-null object
phone             5 non-null object
userId            5 non-null object
dtypes: float64(2), object(8)
memory usage: 480.0+ bytes


In [18]:
ds_dettagli.to_csv("./kijiji_dettagli.csv")